# **Libraries**

In [ ]:
# Basic packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import seaborn as sns
import os, pickle, re
from zipfile import ZipFile
from warnings import filterwarnings

# Machine learning packages
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow import keras

# Performance metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split, TimeSeriesSplit

filterwarnings('ignore')

In [ ]:
window = 50                # Size of array
predictionHorizonMax = 12  # Maximum prediction horizon
neurons = 8               # Number of neurons in LSTM
epochs = 15                 # Number of epochs
batch_size = 64            # Batch size


#  **Functions**

In [ ]:
def SerieMatriz(timeSerie, predictionHorizonMax, window):
  timeSerie = np.squeeze(timeSerie)

  X = np.zeros([len(timeSerie)-predictionHorizonMax-window+1, window])
  y = np.zeros([len(timeSerie)-predictionHorizonMax-window+1, predictionHorizonMax])
  for i in range(X.shape[0]):
      X[i,:] = timeSerie[i:i+window]
      y[i,:] = timeSerie[i+window: i+window+predictionHorizonMax]
  
  return X, np.squeeze(y)

# **Read datasets**

In [ ]:
path = '/Users/guane/Documentos/Doctorate/GuajiraSustainableWindBot/data/raw'
datasets = os.listdir(path)

data = {}
for csv_file in datasets:

    name = csv_file.split('.')[0]
    name = re.sub(r'open_meteo_', '', name)
  
    print("======"*8, name)
  
    # Read dataset
    df = pd.read_csv(path + '/' + csv_file)

    timeSerie = df['wind_speed_10m']
    time = np.arange(0, len(timeSerie),1)

    # Fill missing values
    valueMax = timeSerie.max()
    mean_value = timeSerie.mean()
    timeSerie_filled = timeSerie.fillna(mean_value)

    timeSerieNormalized = timeSerie_filled / valueMax

    # Create time series matrix
    X, y = SerieMatriz(timeSerieNormalized, predictionHorizonMax, window)

    X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_valid, y_train, y_valid = train_test_split(X_, y_, test_size=0.2, random_state=42)

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_valid = np.reshape(X_valid, (X_valid.shape[0], X_valid.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    data[name] = {}
    data[name]['X_train'] = X_train
    data[name]['X_valid'] = X_valid
    data[name]['X_test'] = X_test
    data[name]['y_train'] = y_train*valueMax
    data[name]['y_valid'] = y_valid*valueMax
    data[name]['y_test'] = y_test*valueMax
    data[name]['timeSerie'] = timeSerie_filled
    data[name]['mean_value'] = mean_value
    data[name]['valueMax'] = valueMax

    # Plot dataset
    plt.figure(figsize=(35,5))
    plt.plot(time[0:X_train.shape[0]], timeSerie[0:X_train.shape[0]], 'g', label= 'Training, samples')
    plt.plot(time[X_train.shape[0]:X_train.shape[0]+X_valid.shape[0]], timeSerie[X_train.shape[0]:X_train.shape[0]+X_valid.shape[0]], 'b', label= 'Validation')
    plt.plot(time[X_train.shape[0]+X_valid.shape[0]:], timeSerie[X_train.shape[0]+X_valid.shape[0]:], 'r', label= 'Test')
    plt.title('Dataset', c='r')
    plt.xticks(c='r')
    plt.yticks(c='r')
    plt.xlim(0,timeSerie.shape[0])
    plt.legend()
    plt.grid(True)
    plt.show()

    # save dataset
    with open(f'/Users/guane/Documentos/Doctorate/GuajiraSustainableWindBot/models/LSTM/{name}_dataset.pkl', 'wb') as f:
        pickle.dump(data, f)
  


# **Prediction model**

In [ ]:
def LSTM_model(X_train, y_train, X_valid, y_valid, X_test, y_test):
    
    model_LSTM = keras.models.Sequential([
    keras.layers.LSTM(neurons, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(neurons, return_sequences=True),
    keras.layers.LSTM(neurons),
    keras.layers.Dense(predictionHorizonMax)
    ])

    return model_LSTM
    

In [ ]:
for name in data.keys():

    print("======"*8, name)
    # Get data
    X_train = data[name]['X_train']
    y_train = data[name]['y_train']
    X_valid = data[name]['X_valid']
    y_valid = data[name]['y_valid']
    X_test = data[name]['X_test']
    y_test = data[name]['y_test']

    # Create model
    model = LSTM_model(X_train, y_train, X_valid, y_valid, X_test, y_test)
    #model.summary()

    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse', 'mape'])

    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid), verbose=0)

    # Plot training & validation loss values
    plt.figure()
    plt.subplot(1, 2, 1)
    plt.title(f"Model loss - {name}")
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.xlabel('Epoch'); plt.ylabel('Loss')
    plt.legend(['train', 'validation'])
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'])
    plt.plot(history.history['mape'])
    plt.plot(history.history['r2'])
    plt.xlabel('Epoch'); plt.ylabel('Loss')
    plt.legend(['mae', 'mape'])
    plt.show()

    # save model
    model.save(f'/Users/guane/Documentos/Doctorate/GuajiraSustainableWindBot/models/LSTM/{name}_model.keras')

    # save history
    with open(f'/Users/guane/Documentos/Doctorate/GuajiraSustainableWindBot/models/LSTM/{name}_history.pkl', 'wb') as f:
        pickle.dump(history.history, f)
